## Imports

In [1]:
import torch
use_gpu = torch.cuda.is_available()
print('Use GPU: {}'.format(use_gpu))

use_mps = torch.backends.mps.is_available()
print('Use MPS: {}'.format(use_mps))
from train_dqn import *
agent = PongAgent()

## What is Reinforcement Learning? AI Agents

![RL](./images/rl_overview.png "Reinforcement Learning Overview")

## What is DQN? Deep Q Networks:

![DQN](./images/dqn.png "DQN AI Model Architecture")

## Questions: What is RL in pong?

![PONG](./images/pong_gif.gif "ATARI PONG Environment")

## Untrained Model

In [6]:
agent.play(2)

/Users/jesse/miniconda3/envs/pong_dqn/lib/python3.9/site-packages/gym/logger.py:30: UserWarning: WARN: <class 'atari_wrappers.FrameStack'> doesn't implement 'reset' method, but it implements deprecated '_reset' method.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## Barely Trained Model: Might be even worse than the initial model! Why? Trained on nearly random behavior!

In [8]:
agent.load_model('./model/current_model_50.pth')
agent.play(2)

In [9]:
agent.load_model('./model/current_model_150.pth')
agent.play(2)

AGX: exceeded compiled variants footprint limit


In [ ]:
agent.close_env()

## What more can reinforcement learning do?